<a href="https://colab.research.google.com/github/wcj365/python-stats-dataviz/blob/master/examples/laplace_mechanism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# LaPlace Mechanism for Differential Privacy

https://stackoverflow.com/questions/40694682/implementation-techinque-for-differential-privacy

In [77]:
import numpy as np
import pandas as pd

In [22]:
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [23]:
header_string = """
age: continuous.
workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
fnlwgt: continuous.
education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
education-num: continuous.
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
sex: Female, Male.
capital-gain: continuous.
capital-loss: continuous.
hours-per-week: continuous.
native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
"""

In [24]:
headers = header_string.split("\n")
headers

['',
 'age: continuous.',
 'workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.',
 'fnlwgt: continuous.',
 'education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.',
 'education-num: continuous.',
 'marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.',
 'occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.',
 'relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.',
 'race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.',
 'sex: Female, Male.',
 'capital-gain: continuous.',
 'capital-loss: continuous.',
 'hours-per-week: continuous.',
 '

In [25]:
headers = headers[1: -1]
headers

['age: continuous.',
 'workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.',
 'fnlwgt: continuous.',
 'education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.',
 'education-num: continuous.',
 'marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.',
 'occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.',
 'relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.',
 'race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.',
 'sex: Female, Male.',
 'capital-gain: continuous.',
 'capital-loss: continuous.',
 'hours-per-week: continuous.',
 'nativ

In [26]:
header_names = []
for header in headers:
    name = header.split(":")[0]
    header_names.append(name)

print(header_names)

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']


In [27]:
len(header_names)

14

In [28]:
header_names.append("income")
df.columns = header_names
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [29]:
df.sample(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
23576,23,Private,91842,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,30,United-States,<=50K
23777,47,Private,33865,Some-college,10,Divorced,Exec-managerial,Not-in-family,White,Female,0,0,48,United-States,<=50K
858,34,Private,188798,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Female,0,0,40,United-States,<=50K
4665,35,Private,332588,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,0,30,United-States,<=50K
29270,47,Private,225065,5th-6th,3,Separated,Sales,Unmarried,White,Female,0,0,40,Mexico,<=50K
6035,78,Self-emp-not-inc,316261,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Male,99999,0,20,United-States,>50K
29399,21,Private,265356,Some-college,10,Never-married,Sales,Other-relative,White,Male,0,0,40,United-States,<=50K
27831,54,Private,511668,Doctorate,16,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,43,United-States,>50K
6613,23,Private,221955,9th,5,Never-married,Handlers-cleaners,Other-relative,White,Male,0,0,39,Mexico,<=50K
10559,28,Private,388112,1st-4th,2,Never-married,Farming-fishing,Unmarried,White,Male,0,0,77,Mexico,<=50K


In [30]:
df.describe(include="all")

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
count,32561.000000,32561,3.256100e+04,32561,32561.000000,32561,32561,32561,32561,32561,32561.000000,32561.000000,32561.000000,32561,32561
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,22696,NaN,10501,NaN,14976,4140,13193,27816,21790,NaN,NaN,NaN,29170,24720
mean,38.581647,NaN,1.897784e+05,NaN,10.080679,NaN,NaN,NaN,NaN,NaN,1077.648844,87.303830,40.437456,NaN,NaN
std,13.640433,NaN,1.055500e+05,NaN,2.572720,NaN,NaN,NaN,NaN,NaN,7385.292085,402.960219,12.347429,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.178270e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.783560e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.370510e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


In [37]:
df["income"].unique()

array([' <=50K', ' >50K'], dtype=object)

In [39]:
df[df["income"] == " <=50K"].shape

(24720, 15)

In [48]:
df["income2"] = df["income"].apply(lambda x:x.strip())
df["income2"].unique()

array(['<=50K', '>50K'], dtype=object)

In [58]:
df_50kplus = df[df["income2"] == ">50K"]
df_50kplus.shape

(7841, 16)

In [68]:
df_groups = df_50kplus.groupby("race").count()
df_groups

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,income,income2
race,,,,,,,,,,,,,,,
Amer-Indian-Eskimo,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36
Asian-Pac-Islander,276,276,276,276,276,276,276,276,276,276,276,276,276,276,276
Black,387,387,387,387,387,387,387,387,387,387,387,387,387,387,387
Other,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25
White,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117


In [69]:
df_groups = df_groups.reset_index() 
df_groups

,race,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,income,income2
0,Amer-Indian-Eskimo,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36
1,Asian-Pac-Islander,276,276,276,276,276,276,276,276,276,276,276,276,276,276,276
2,Black,387,387,387,387,387,387,387,387,387,387,387,387,387,387,387
3,Other,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25
4,White,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117,7117


In [70]:
df_groups = df_groups[df_groups.columns[:2]]
df_groups

,race,age
0,Amer-Indian-Eskimo,36
1,Asian-Pac-Islander,276
2,Black,387
3,Other,25
4,White,7117


In [72]:
df_groups.rename(columns={"age":"count"}, inplace=True)
df_groups

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,race,count
0,Amer-Indian-Eskimo,36
1,Asian-Pac-Islander,276
2,Black,387
3,Other,25
4,White,7117


In [75]:
def laplace_mechanism(x, epsilon):
    x +=  np.random.laplace(0, 1.0/epsilon, 1)[0]
    return x

In [84]:
df_groups["count2"] = df_groups["count"].apply(laplace_mechanism, args=(0.01,))
df_groups

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,race,count,count2
0,Amer-Indian-Eskimo,36,87.975787
1,Asian-Pac-Islander,276,189.710908
2,Black,387,566.150808
3,Other,25,76.985404
4,White,7117,6802.952361
